# This notebook

In [4]:
#  Initialization
import os
from datetime import datetime as dt
import numpy as np
from matplotlib import pyplot as plt
import torch
import torch.nn as nn
from torch import optim
from pytorch_fid import fid_score
from tqdm import tqdm
import logging
import argparse
#import wandb
# Custom Modules
from utils import *
from modules import UNet
from DDPM import *

# MNIST

## Load trained model

In [5]:
model = UNet(c_in=1, c_out=1,img_dim=28,initial_feature_maps=64,num_max_pools=2)
ckpt = torch.load(f"models/mnist_mode.pt", map_location=torch.device('cpu'))
model.load_state_dict(ckpt)

<All keys matched successfully>

In [6]:
diffusion = DiffusionProcess(img_shape=(1,32,32), img_size=32)
x = diffusion.sampling(model, num_img= 1)

AssertionError: Torch not compiled with CUDA enabled